In [2]:
import os
import random
from skimage import data
import skimage.transform
import matplotlib
import numpy as np
import tensorflow as tf
from numpy import array
import h5py
import math
from keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D
from keras.models import Model
from keras.initializers import glorot_uniform
from keras.utils import np_utils
import matplotlib.pyplot as plt
from sklearn.cross_validation import train_test_split

import keras.backend as K
K.set_image_data_format('channels_last')
K.set_learning_phase(1)


Using TensorFlow backend.
/home/ubuntu/anaconda3/envs/tf/lib/python3.5/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [3]:
def identity_block(X, f, filters, stage, block):
    """
    Implements the identity block
    Arguments:
    X -- input tensor of shape (m, n_H_prev, n_W_prev, n_C_prev)
    f -- integer, specifying the shape of the middle CONV's window for the main path
    filters -- python list of integers, defining the number of filters in the CONV layers of the main path
    stage -- integer, used to name the layers, depending on their position in the network
    block -- string/character, used to name the layers, depending on their position in the network
    Returns:
    X -- output of the identity block, tensor of shape (m, n_H, n_W, n_C)
    """
    
    # defining name base
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'
    
    # Retrieve Filters
    F1, F2, F3 = filters
    
    X_shortcut = X
    X = Conv2D(filters=F1, kernel_size=(1, 1), strides=(1, 1), padding="VALID", name=conv_name_base + '2a',
               kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name=bn_name_base + '2a')(X)
    X = Activation('relu')(X)
    
    # Second component of main path
    X = Conv2D(filters=F2, kernel_size=(f, f), strides=(1, 1), padding="SAME", name=conv_name_base + '2b',
               kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name=bn_name_base + '2b')(X)
    X = Activation('relu')(X)

    # Third component of main path
    X = Conv2D(filters=F3, kernel_size=(1, 1), strides=(1, 1), padding="VALID", name=conv_name_base + '2c',
               kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name=bn_name_base + '2c')(X)

    # Adding shortcut value to main path, and passing it through a RELU activation
    X = Add()([X, X_shortcut])
    X = Activation('relu')(X)

    return X

In [4]:
def convolutional_block(X, f, filters, stage, block, s=2):
    """
    Implements convolutional block
    Arguments:
    X -- input tensor of shape (m, n_H_prev, n_W_prev, n_C_prev)
    f -- integer, specifying the shape of the middle CONV's window for the main path
    filters -- python list of integers, defining the number of filters in the CONV layers of the main path
    stage -- integer, used to name the layers, depending on their position in the network
    block -- string/character, used to name the layers, depending on their position in the network
    s -- Integer, specifying the stride to be used
    Returns:
    X -- output of the convolutional block, tensor of shape (m, n_H, n_W, n_C)
    """

    # defining name basis
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'
    # Retrieve Filters
    F1, F2, F3 = filters

    # Save the input value
    X_shortcut = X

    # First component of main path
    X = Conv2D(F1, (1, 1), strides=(s, s), name=conv_name_base + '2a', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name=bn_name_base + '2a')(X)
    X = Activation('relu')(X)

    # Second component of main path
    X = Conv2D(filters=F2, kernel_size=(f, f), strides=(1, 1), padding="SAME", name=conv_name_base + '2b',
               kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name=bn_name_base + '2b')(X)
    X = Activation('relu')(X)

    # Third component of main path
    X = Conv2D(filters=F3, kernel_size=(1, 1), strides=(1, 1), padding="VALID", name=conv_name_base + "2c",
               kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name=bn_name_base + '2c')(X)

    # Shortcut path -> Use filters of layer 3 and strides of layer 1
    X_shortcut = Conv2D(filters=F3, kernel_size=(1, 1), strides=(s, s), padding="VALID", name=conv_name_base + '1',
                        kernel_initializer=glorot_uniform(seed=0))(X_shortcut)
    X_shortcut = BatchNormalization(axis=3, name=bn_name_base + '1')(X_shortcut)

    # Adding shortcut value to main path, and passing it through a RELU activation
    X = Add()([X, X_shortcut])
    X = Activation('relu')(X)

    return X

In [5]:
def ResNet50(input_shape=(64, 64, 3), classes=6):
    """
    Implementation of the popular ResNet50 the following architecture:
    CONV2D -> BATCHNORM -> RELU -> MAXPOOL -> CONVBLOCK -> IDBLOCK*2 -> CONVBLOCK -> IDBLOCK*3
    -> CONVBLOCK -> IDBLOCK*5 -> CONVBLOCK -> IDBLOCK*2 -> AVGPOOL -> TOPLAYER
    Arguments:
    input_shape -- shape of the images of the dataset
    classes -- integer, number of classes
    Returns:
    model -- a Model() instance in Keras
    """

    # Define the input as a tensor with shape input_shape
    X_input = Input(input_shape)

    # Zero-Padding
    X = ZeroPadding2D((3, 3))(X_input)
    
    nb_filters=64

    # Stage 1
    X = Conv2D(nb_filters, (7, 7), strides=(2, 2), name='conv1', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name='bn_conv1')(X)
    X = Activation('relu')(X)
    X = MaxPooling2D((3, 3), strides=(2, 2))(X)

    # Stage 2
    X = convolutional_block(X, f=3, filters=[64, 64, 256], stage=2, block='a', s=1)
    X = identity_block(X, 3, [64, 64, 256], stage=2, block='b')
    X = identity_block(X, 3, [64, 64, 256], stage=2, block='c')

    # Stage 3
    X = convolutional_block(X=X, filters=[128, 128, 512], f=3, s=2, block='a', stage=3)
    X = identity_block(X, f=3, filters=[128, 128, 512], stage=3, block='b')
    X = identity_block(X, f=3, filters=[128, 128, 512], stage=3, block='c')
    X = identity_block(X, f=3, filters=[128, 128, 512], stage=3, block='d')

    # Stage 4
    X = convolutional_block(X=X, filters=[256, 256, 1024], f=3, s=2, block='a', stage=4)
    X = identity_block(X, f=3, filters=[256, 256, 1024], stage=4, block='b')
    X = identity_block(X, f=3, filters=[256, 256, 1024], stage=4, block='c')
    X = identity_block(X, f=3, filters=[256, 256, 1024], stage=4, block='d')
    X = identity_block(X, f=3, filters=[256, 256, 1024], stage=4, block='e')
    X = identity_block(X, f=3, filters=[256, 256, 1024], stage=4, block='f')

    # Stage 5
    X = convolutional_block(X=X, filters=[512, 512, 2048], f=3, s=2, block='a', stage=5)
    X = identity_block(X, f=3, filters=[512, 512, 2048], stage=5, block='b')
    X = identity_block(X, f=3, filters=[512, 512, 2048], stage=5, block='c')

    # average pool
    X = AveragePooling2D(pool_size=(2, 2), name='avg_pool')(X)

    # output layer
    X = Flatten()(X)
    X = Dense(classes, activation='softmax', name='fc' + str(classes), kernel_initializer=glorot_uniform(seed=0))(X)

    # Create model
    model = Model(inputs=X_input, outputs=X, name='ResNet50')

    return model

### read file from one folder



IMG_WIDTH = 759
IMG_HEIGHT = 1012


def load_data(path):
    #print('file_name: ',path)
    images = []
    #file_names = sorted(os.listdir(path)) 
    file_names = sorted(os.listdir(path),key=lambda x: int(os.path.splitext(x)[0]))
    for f in file_names:
        #print('Image_file_name: ',f)
        images.append(data.imread(os.path.join(path, f)))
    return images

def load_label(filename):    
    labels = []
    file = open(filename, 'r') 
    for line in file:
        #print(line)
        labels.append(line.strip('\n'))
    print('label_file_name written: ',filename)
    file.close()
    return labels
    

path = '/home/ubuntu/ttr/TicketToRide'
labelfilename='threeTestSegment.txt'
train_data_directory=os.path.join(path,'Just3')
labelfile=os.path.join(path,labelfilename)
    
images = load_data(train_data_directory)
labels=load_label(labelfile)

print("Unique Labels: {0}\nTotal Images: {1}".format(len(set(labels)), len(images)))


# Resize images
images32 = [skimage.transform.resize(image, (IMG_WIDTH, IMG_HEIGHT))
                for image in images]
#display_images_and_labels(images32, labels)

for image in images32[:5]:
    print("shape: {0}, min: {1}, max: {2}".format(image.shape, image.min(), image.max()))


labels_a = np.array(labels)
images_a = np.array(images32)
print("labels: ", labels_a.shape, "\nimages: ", images_a.shape)



### read from different folder

In [ ]:
def load_data_labels(data_directory):
    #print('file_name: ',path)
    images = []
    labels = []
    #file_names = sorted(os.listdir(path)) 
    
    directories = [d for d in os.listdir(data_directory) 
                   if os.path.isdir(os.path.join(data_directory, d))]
    for d in directories:
        label_value=d.split('class_')[1]
        print(label_value)
        label_directory = os.path.join(data_directory, d)
        
        for f in os.listdir(label_directory):
            images.append(data.imread(os.path.join(label_directory, f)))
            labels.append(label_value)
            
    return images,labels

path = '/home/ubuntu/datasets'
train_data_directory=os.path.join(path,'just3')
    
images, labels = load_data_labels(train_data_directory)
print("Unique Labels: {0}\nTotal Images: {1}".format(len(set(labels)), len(images)))
labels_a = np.array(labels)
images_a = np.array(images)
print("labels: ", labels_a.shape, "\nimages: ", images_a.shape)



0


In [7]:
nb_classes=len(set(labels_a))
X_train, X_test, y_train, y_test = train_test_split(images_a, labels_a, test_size=0.33, random_state=42)
print("Train: ", X_train.shape, "\nTest: ", X_test.shape,"\nnb_classes: ", nb_classes)

Train:  (93, 759, 1012, 3) 
Test:  (46, 759, 1012, 3) 
nb_classes:  3


## one hot encoding 

In [8]:
uniques, id_train=np.unique(y_train,return_inverse=True)
Y_train=np_utils.to_categorical(id_train,nb_classes)

uniques, id_test=np.unique(y_test,return_inverse=True)
Y_test=np_utils.to_categorical(id_test,nb_classes)


In [11]:
# generate model
model = ResNet50(input_shape = (IMG_WIDTH, IMG_HEIGHT, 3), classes = nb_classes)

# compile model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# fit model
model_history = model.fit(X_train, Y_train, epochs = 2, batch_size = 5)

# plot accuracy/loss
#plot_model_history(model_history)

# evaluate model
preds = model.evaluate(X_test, Y_test)
print ("Loss = " + str(preds[0]))
print ("Test Accuracy = " + str(preds[1]))

Epoch 1/2
93/93 [==============================] - 66s 714ms/step - loss: 10.5213 - acc: 0.3011
Epoch 2/2
46/46 [==============================] - 12s 259ms/step
Loss = 10.1614081341
Test Accuracy = 0.369565219983
